In [1]:
import pid_with_genome as prog
from CMAES import CMAES, CMAESVarSet
import numpy as np
import random

c:\Users\kenbl\Desktop\comma\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DECISION_VARS = CMAESVarSet({
    "kp": [-0.5, 0.5],
    "ki": [-0.5, 0.5],
    "kd": [-0.5, 0.5],
    "kdd": [-0.5, 0.5],
})

round_vars_to_nearest = None
# round_vars_to_nearest = 1 / (int(args.discrete_bins) - 1)

In [3]:
rnd = random.Random(42)

In [4]:
nsamples = 48
def fitness(genome):
    files_to_test = [rnd.randrange(0, 20000 - 1) for _ in range(nsamples)]
    # genome = (0.12, 0.115, 0.005, -0.0008)
    results = prog.test_genome(genome, files_to_test, max_workers=24, tqdm=False)
    # print(results)
    return np.mean([result['total_cost'] for result in results])

In [5]:
cmaes = CMAES(
    fitness,
    dvars=DECISION_VARS,
    # num_processes=args.processes,
    show_each_step=False,
    target=0,
    # experiment=exp,
    max_iters=2000,
    pop_size=20,
    # round_to_every=round_vars_to_nearest,
)

cmaes.minimize()

(10_w,20)-aCMA-ES (mu_w=5.9,w_1=27%) in dimension 4 (seed=1, Sat Aug 24 11:40:47 2024)


100%|██████████| 20/20 [01:32<00:00,  4.63s/it]


TypeError: %d format: a real number is required, not NoneType